# Análisis de precios de apartamentos de Bogotá

# Importacion de librerías y funciones auxiliares

In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [52]:
df = pd.read_csv('C:/Users/jmriv/Documents/Ciencia de Datos - Repos/Tutorial regresión/apartamentos.csv')

In [53]:
def ver_datos(data, col_name, categorica=False):
    print(data[col_name].describe())
    print('--'*10)
    print(f'Número de datos nulos: {data[col_name].isnull().sum()}')
    if categorica:
        print('--'*10)
        print(data[col_name].value_counts(dropna=False))

# Caracterización de los datos

In [54]:
# Información básica del dataset
print('Dimensiones del dataset (filas, columnas):', df.shape)
print('--'*10)
print('Tipos de datos:')
display(df.info())

Dimensiones del dataset (filas, columnas): (43013, 46)
--------------------
Tipos de datos:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43013 entries, 0 to 43012
Data columns (total 46 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   _id                      43013 non-null  object 
 1   codigo                   43013 non-null  object 
 2   tipo_propiedad           43013 non-null  object 
 3   tipo_operacion           43013 non-null  object 
 4   precio_venta             27584 non-null  float64
 5   area                     43013 non-null  float64
 6   habitaciones             43012 non-null  float64
 7   banos                    43012 non-null  float64
 8   administracion           35088 non-null  float64
 9   parqueaderos             43012 non-null  float64
 10  sector                   41372 non-null  object 
 11  estrato                  43012 non-null  float64
 12  antiguedad               43003 non-nul

None

In [55]:
print('Estadísticas descriptivas:')
print(df.describe())

Estadísticas descriptivas:


       precio_venta          area  habitaciones         banos  administracion  \
count  2.758400e+04  4.301300e+04  43012.000000  43012.000000    3.508800e+04   
mean   2.364794e+09  1.800341e+02      2.615340      2.701339    4.059551e+06   
std    5.574198e+10  9.163002e+03      0.850046      1.196659    6.781642e+07   
min    1.000000e+06  0.000000e+00      1.000000      0.000000    1.000000e+00   
25%    4.029000e+08  6.300000e+01      2.000000      2.000000    3.490000e+05   
50%    7.000000e+08  1.000000e+02      3.000000      3.000000    6.500000e+05   
75%    1.330000e+09  1.740000e+02      3.000000      4.000000    1.200000e+06   
max    4.250000e+12  1.900000e+06      7.000000      6.000000    3.500000e+09   

       parqueaderos       estrato       latitud      longitud  \
count  43012.000000  43012.000000  43013.000000  43013.000000   
mean       1.662862      4.844648      4.686099    -74.062808   
std        1.114685      1.236190      0.038297      0.033544   
min       

Se identifica que la variable _Parqueaderos_ tiene una entrada negativa, por lo que se revisará su validez

In [56]:
df.sample(2)

,_id,codigo,tipo_propiedad,tipo_operacion,precio_venta,area,habitaciones,banos,administracion,parqueaderos,...,vigilancia,coords_modified,localidad,barrio,estacion_tm_cercana,distancia_estacion_tm_m,is_cerca_estacion_tm,parque_cercano,distancia_parque_m,is_cerca_parque
21654,66d86d2eceda690e8550dbf6,851-M1229945,APARTAMENTO,VENTA,460000000.0,55.0,1.0,2.0,500000.0,2.0,...,1.0,False,USAQUEN,SANTA BIBIANA,Calle 106,512.53,0,PARQUE VECINAL URBANIZACIÓN NORMANDIA I II II ...,1663.39,0
4202,66d86c8eceda690e855097ca,MC4968764,APARTAMENTO,VENTA,195000000.0,47.0,2.0,1.0,160800.0,0.0,...,0.0,False,USAQUEN,TIBABITA EL REPOSO,Terminal,1552.99,0,PARQUE VECINAL DESARROLLO VERBENAL I,1141.20,0


In [57]:
# Información básica del dataset
print('Dimensiones del dataset (filas, columnas):', df.shape)
print('\nTipos de datos:')
print(df.dtypes)
print('\nEstadísticas descriptivas:')
print(df.describe())

Dimensiones del dataset (filas, columnas): (43013, 46)

Tipos de datos:
_id                         object
codigo                      object
tipo_propiedad              object
tipo_operacion              object
precio_venta               float64
area                       float64
habitaciones               float64
banos                      float64
administracion             float64
parqueaderos               float64
sector                      object
estrato                    float64
antiguedad                  object
latitud                    float64
longitud                   float64
direccion                   object
descripcion                 object
website                     object
last_view                   object
datetime                    object
url                         object
timeline                    object
estado                      object
compañia                    object
precio_arriendo            float64
jacuzzi                    float64
piso              

In [58]:
df['tipo_operacion'].value_counts()

tipo_operacion
VENTA               27270
ARRIENDO            15515
VENTA Y ARRIENDO      228
Name: count, dtype: int64

In [59]:
df[['tipo_propiedad','tipo_operacion']].value_counts()

tipo_propiedad             tipo_operacion  
APARTAMENTO                VENTA               27200
                           ARRIENDO            15515
                           VENTA Y ARRIENDO      228
CASA CON CONJUNTO CERRADO  VENTA                  60
CASA                       VENTA                  10
Name: count, dtype: int64

## Validez

In [60]:
# Se asume que el valor negativo de los parqueaderos puede ser un typo
df.loc[df['parqueaderos'] < 0, 'parqueaderos'] = 2

In [137]:
# Se asume que los inmuebles con 10, 20 y 30 parqueaderos también fueron typos
df['parqueaderos'].value_counts()

parqueaderos
2.0    15528
1.0    12680
0.0     6615
3.0     5046
4.0     3143
Name: count, dtype: int64

In [136]:
df['parqueaderos'] = df['parqueaderos'].replace({10:1,20:2,30:3})

In [140]:
df['banos'].value_counts()

banos
2.0    14825
3.0    10605
4.0     6615
1.0     6614
5.0     4329
6.0        1
Name: count, dtype: int64

In [139]:
# Se considera como no valido los inmuebles que no tiene baños
df = df[df['banos'] > 0]

In [80]:
# Dado que no hay ningún inmueble con salón comunal, se excluye del análisis
df['salon_comunal'].value_counts()

salon_comunal
0.0    43011
Name: count, dtype: int64

In [100]:
# Dado que no hay ningún inmueble que permita mascotas, se excluye del análisis
df['permite_mascotas'].value_counts()

permite_mascotas
0.0    43011
Name: count, dtype: int64

In [113]:
# Dado que no hay ningún inmueble que tenga chimenea, se excluye del análisis
df['chimenea'].value_counts()

chimenea
0.0    43011
Name: count, dtype: int64

## Completitud
Se eliminarán algunas variables teniendo en cuenta el número de nulos que presentan

In [141]:
print('\nValores nulos por columna:')
print(df.isnull().sum().sort_values(ascending=False))


Valores nulos por columna:
precio_arriendo            27170
precio_venta               15420
timeline                   13581
administracion              7920
compañia                    4529
sector                      1638
estado                       795
barrio                       193
antiguedad                    10
website                        2
salon_comunal                  2
terraza                        2
vigilancia                     2
piscina                        2
ascensor                       2
conjunto_cerrado               2
gimnasio                       2
last_view                      2
datetime                       2
jacuzzi                        2
chimenea                       2
permite_mascotas               2
estrato                        1
tipo_operacion                 0
_id                            0
codigo                         0
parqueaderos                   0
banos                          0
habitaciones                   0
area           

Se deciden excluir las siguientes variables debido al número de datos nulos que presentan:
- Closets
- Piso
- URL
- Dirección

In [62]:
df.drop(columns=['closets', 'piso','url','direccion'], inplace=True)

## Relevancia
Se decide seleccionar un subconjunto de los datos para trabajar con ellos

In [142]:
selected_cols = [
    'precio_arriendo',
    'precio_venta',
    'terraza',
    'vigilancia',
    'piscina',
    'ascensor',
    'conjunto_cerrado',
    'gimnasio',
    'jacuzzi',
    'estrato',
    'habitaciones',
    'banos',
    'parqueaderos',
    'tipo_operacion',
    'tipo_propiedad',
    'area',
    'localidad',
    'distancia_estacion_tm_m',
    'is_cerca_estacion_tm',
    'distancia_parque_m',
    'is_cerca_parque',
    #'antiguedad',
    'tipo_propiedad',
    'tipo_operacion'
    ]

In [143]:
df_clean=df[selected_cols]

In [144]:
print(df_clean.isnull().sum().sort_values(ascending=False))

precio_arriendo            27170
precio_venta               15420
terraza                        2
vigilancia                     2
piscina                        2
ascensor                       2
conjunto_cerrado               2
gimnasio                       2
jacuzzi                        2
estrato                        1
habitaciones                   0
banos                          0
parqueaderos                   0
tipo_operacion                 0
tipo_propiedad                 0
area                           0
localidad                      0
distancia_estacion_tm_m        0
is_cerca_estacion_tm           0
distancia_parque_m             0
is_cerca_parque                0
tipo_propiedad                 0
tipo_operacion                 0
dtype: int64


## Preparación de variables categóricas

Se incluyeron dos columnas categóricas como parte del análisis:
- Localidad
- Antigüedad

Para la variable _localidad_ se dividirá la variable en columnas dummy

In [145]:
df_clean = pd.get_dummies(df_clean, columns=["localidad"])

In [146]:
df_clean.columns

Index(['precio_arriendo', 'precio_venta', 'terraza', 'vigilancia', 'piscina',
       'ascensor', 'conjunto_cerrado', 'gimnasio', 'jacuzzi', 'estrato',
       'habitaciones', 'banos', 'parqueaderos', 'tipo_operacion',
       'tipo_propiedad', 'area', 'distancia_estacion_tm_m',
       'is_cerca_estacion_tm', 'distancia_parque_m', 'is_cerca_parque',
       'tipo_propiedad', 'tipo_operacion', 'localidad_ANTONIO NARINO',
       'localidad_BARRIOS UNIDOS', 'localidad_BOSA', 'localidad_CANDELARIA',
       'localidad_CHAPINERO', 'localidad_CIUDAD BOLIVAR', 'localidad_ENGATIVA',
       'localidad_FONTIBON', 'localidad_KENNEDY', 'localidad_LOS MARTIRES',
       'localidad_PUENTE ARANDA', 'localidad_RAFAEL URIBE URIBE',
       'localidad_SAN CRISTOBAL', 'localidad_SANTA FE', 'localidad_SUBA',
       'localidad_TEUSAQUILLO', 'localidad_TUNJUELITO', 'localidad_USAQUEN',
       'localidad_USME'],
      dtype='object')

Para la variable _antigüedad_ se propone convertirla en 3 variables:
 - Nuevo: incluye las variables _SOBRE PLANOS_, _EN CONSTRUCCION_ y _PARA ESTRENAR_
 - Usado: incluye las variables _ENTRE 0 Y 5 ANOS_ hasta _MAS DE 20 ANOS_
 - Remodelado: incluye la variable _REMODELADO_

In [147]:
df['antiguedad'].value_counts()

antiguedad
MAS DE 20 ANOS        14349
ENTRE 10 Y 20 ANOS    11527
ENTRE 0 Y 5 ANOS       8047
ENTRE 5 Y 10 ANOS      7019
REMODELADO             1811
SOBRE PLANOS            107
EN CONSTRUCCION         101
PARA ESTRENAR            18
Name: count, dtype: int64

In [148]:
#plt.figure(figsize=(12,4))
#ax = sns.boxplot(
#    data=df_clean,
#    x='antiguedad',
#    y='precio_venta',
#    showmeans=True
#)
#plt.ylim(0,3000000000)

# Modelado

1. crear el traint, test y validation
2. Crear al menos dos modelos distintos para predecir el precio_venta
3. Metricas
4. Elegir un modelo y jsutificarlo, oportunidades de mejora
5. interpretabilidad shapley line 
6. generacion de valor 
7. insights

In [149]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

## Modelo exclusivo para apartamentos en venta

In [150]:
# Filtar el dataframe para inmuebles tipo APARTAMENTO y operacion VENTA o VENTA Y ARRIENDO
df_aptos_venta =df_clean.loc[(df["tipo_propiedad"]=="APARTAMENTO") & (df["tipo_operacion"].isin(['VENTA','VENTA Y ARRIENDO']))]
df_aptos_venta = df_aptos_venta.drop(columns=['precio_arriendo','tipo_propiedad', 'tipo_operacion'])
print(f'El dataset tiene {df_aptos_venta.shape[0]} entradas antes de eliminar nulos')
df_aptos_venta.dropna(inplace=True)
print(f'El dataset tiene {df_aptos_venta.shape[0]} entradas después de eliminar nulos')

El dataset tiene 27414 entradas antes de eliminar nulos
El dataset tiene 27411 entradas después de eliminar nulos


In [154]:
for i in df_aptos_venta.columns:
    print(df_aptos_venta[i].value_counts())

precio_venta
1.200000e+09    402
1.100000e+09    367
6.500000e+08    348
1.300000e+09    345
7.500000e+08    335
               ... 
1.610000e+08      1
1.499990e+08      1
1.335000e+08      1
1.825000e+08      1
1.972000e+08      1
Name: count, Length: 2755, dtype: int64
terraza
0.0    27377
1.0       34
Name: count, dtype: int64
vigilancia
1.0    16423
0.0    10988
Name: count, dtype: int64
piscina
0.0    24539
1.0     2872
Name: count, dtype: int64
ascensor
1.0    17873
0.0     9538
Name: count, dtype: int64
conjunto_cerrado
0.0    15570
1.0    11841
Name: count, dtype: int64
gimnasio
0.0    18545
1.0     8866
Name: count, dtype: int64
jacuzzi
0.0    25991
1.0     1420
Name: count, dtype: int64
estrato
6.0    11974
4.0     5752
5.0     5312
3.0     3368
2.0      919
1.0       79
0.0        7
Name: count, dtype: int64
habitaciones
3.0    16418
2.0     5831
4.0     2681
1.0     2189
5.0      291
7.0        1
Name: count, dtype: int64
banos
2.0    9407
3.0    7315
4.0    4725
5.0    30

### Separación en subconjuntos

In [155]:
X_aptos_venta = df_aptos_venta.drop(columns=['precio_venta'])
y_aptos_venta = df_aptos_venta['precio_venta']
print("Las caracteristicas iniciales del dataset son:", X_aptos_venta.columns , "Para describir la variable", y_aptos_venta.name)

Las caracteristicas iniciales del dataset son: Index(['terraza', 'vigilancia', 'piscina', 'ascensor', 'conjunto_cerrado',
       'gimnasio', 'jacuzzi', 'estrato', 'habitaciones', 'banos',
       'parqueaderos', 'area', 'distancia_estacion_tm_m',
       'is_cerca_estacion_tm', 'distancia_parque_m', 'is_cerca_parque',
       'localidad_ANTONIO NARINO', 'localidad_BARRIOS UNIDOS',
       'localidad_BOSA', 'localidad_CANDELARIA', 'localidad_CHAPINERO',
       'localidad_CIUDAD BOLIVAR', 'localidad_ENGATIVA', 'localidad_FONTIBON',
       'localidad_KENNEDY', 'localidad_LOS MARTIRES',
       'localidad_PUENTE ARANDA', 'localidad_RAFAEL URIBE URIBE',
       'localidad_SAN CRISTOBAL', 'localidad_SANTA FE', 'localidad_SUBA',
       'localidad_TEUSAQUILLO', 'localidad_TUNJUELITO', 'localidad_USAQUEN',
       'localidad_USME'],
      dtype='object') Para describir la variable precio_venta


In [156]:
X_train, X_test, y_train, y_test = train_test_split(X_aptos_venta, y_aptos_venta, test_size=0.30, random_state=42)

In [157]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [164]:
X_train_scaled

array([[-0.03957281,  0.8127958 , -0.33969627, ..., -0.02797125,
         1.32557498, -0.05361683],
       [-0.03957281, -1.23032132, -0.33969627, ..., -0.02797125,
         1.32557498, -0.05361683],
       [-0.03957281, -1.23032132, -0.33969627, ..., -0.02797125,
        -0.75438962, -0.05361683],
       ...,
       [-0.03957281,  0.8127958 ,  2.9438062 , ..., -0.02797125,
        -0.75438962, -0.05361683],
       [-0.03957281,  0.8127958 , -0.33969627, ..., -0.02797125,
        -0.75438962, -0.05361683],
       [-0.03957281,  0.8127958 , -0.33969627, ..., -0.02797125,
         1.32557498, -0.05361683]])

## Entrenamiento de los modelos

In [170]:
from sklearn.model_selection import GridSearchCV, StratifiedKFold, cross_val_score
from sklearn.linear_model import LinearRegression, Ridge

#### Regresión lineal

In [171]:
# Para la divisón en entrenamiento y validación se dividirá el conjunto de entrenamiento mediante búsqueda estratificada
cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
model = LinearRegression()
scores = cross_val_score(model, X_train_scaled, y_train, cv=cv, scoring='neg_mean_squared_error')

c:\Users\jmriv\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:811: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


In [173]:
rmse_scores = np.sqrt(-scores)

print("RMSE per fold:", rmse_scores)
print("Average RMSE:", rmse_scores.mean())

RMSE per fold: [4.78820536e+10 8.36164930e+10 5.23749791e+10 2.55120839e+10
 8.46963699e+10]
Average RMSE: 58816395916.95107
